In [ ]:
#libraries for data manipulation
import pandas as pd
import numpy as np

#libraries for visualizations
import matplotlib.pyplot as plt
import seaborn as sns 

%matplotlib inline

### Tomamos el dataset limpio y le agregamos las columnas creadas para el análisis del TP1

In [ ]:
#Clean data
tp1CSV= pd.read_csv('./ToChangeKeywordsAndLocations/withoutEncoding.csv')
tp1CSV.head()

In [ ]:
#Adding tweet's length information 
tp1CSV['tweet_length'] = tp1CSV.text.str.len()
tp1CSV.head()

In [ ]:
def getter(text, char):
    pos = text.find(char)
    text = text[pos:]
    #Some users or hashtags finish with : or .
    if text.endswith(':') or text.endswith('.'):
        text = text[:-1]
    return text

In [ ]:
#Adding amount of hasthag, users and link per tweet

def validUser(userName):
    if '@' in userName:
        user = getter(userName, '@')
        length = len(user)
        if (length > 1 and length <= 16):
            for char in user[1:]:
                if not(char.isalnum() or char == '_'): return False
            return True
    return False

In [ ]:
def validLink(link):
    type1 = 'https://'
    type2 = 'http://'
    if type1 in link and len(link) > 9: return True
    if type2 in link and len(link) > 8: return True
    return False

In [ ]:
def validHashtag(hashtag):
    if '#' in hashtag:
        hashtag = getter(hashtag, '#')
        hashtag = hashtag[1:]
        return hashtag.isalnum()
    return False

In [ ]:
def analyzeTweets(text):
    result = [0,0,0] #number of usersTagged, hashtags and links
    text = text.split()
    for word in text:
        if validUser(word): result[0] += 1
        elif validHashtag(word): result[1] += 1
        elif validLink(word): result[2] += 1
    return result

In [ ]:
def getInfo(dataList, pos):
    return dataList[pos]

In [ ]:
tp1CSV['aux_column'] = tp1CSV.text.apply(analyzeTweets)

tp1CSV['users_tagged'] = tp1CSV.aux_column.apply(getInfo,args=(0,))
tp1CSV['hashtags'] = tp1CSV.aux_column.apply(getInfo,args=(1,))
tp1CSV['links'] = tp1CSV.aux_column.apply(getInfo,args=(2,))

del tp1CSV['aux_column']
tp1CSV.head()

In [ ]:
#Casting categorical data

tp1CSV['keyword'] = tp1CSV['keyword'].astype('category')
tp1CSV['location'] = tp1CSV['location'].astype('category')  

In [ ]:
#Encoding categorical data

tp1CSV['keyword'] = tp1CSV['keyword'].cat.codes
tp1CSV['location'] = tp1CSV['location'].cat.codes
tp1CSV.head()

### Analizamos relación entre las variables categóricas y el target

In [ ]:
corr = tp1CSV.corr()
ax = sns.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
);
ax.set_title('Correlation matrix', fontsize = 18)